Android gameplay footage (Hill Climb Racing) via scrcpy → virtual cam → Python



-------
# Running the model
-------

In [1]:
import cv2
import subprocess
import time
import torch
import numpy as np
from IPython.core.display_functions import clear_output

In [2]:
# === CONFIG ===
ACCEL_POS = (200, 850, 240, 850, 500)  #(x1, y1, x2, y2)
BRAKE_POS = (2115, 850, 2200, 850, 500)  #(x1, y1, x2, y2)
VIDEO_DEVICE = '/dev/video10'
HOLD_DURATION_MS = 10_000  # 10 seconds (long enough to cover many steps)
TAP_DURATION_MS = 1  # 10 seconds (long enough to cover many steps)

....

In [3]:
# === HELPERS ===
def start_hold(x, y, duration_ms=HOLD_DURATION_MS):
    """Start a long press in background (non-blocking)"""
    return subprocess.Popen([
        'adb', 'shell', 'input', 'swipe',
        str(x), str(y), str(x), str(y), str(duration_ms)
    ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)


def stop_hold(proc):
    """Gracefully stop a running hold"""
    if proc is not None:
        proc.terminate()
        try:
            proc.wait(timeout=0.1)
        except subprocess.TimeoutExpired:
            proc.kill()


def get_action_from_model(frame):
    """
    Replace this with your PyTorch model.
    Must return one of: "accel", "brake", "none"
    """
    # Placeholder: random action (replace with real inference)
    time.sleep(0.02)  # Simulate model latency (~20ms)
    return np.random.choice(["accel", "brake", "none"])


def get_frame(cap):
    """Grab single frame from video device"""
    ret, frame = cap.read()
    if not ret:
        raise RuntimeError("Failed to read frame")
    return frame


def resize_img(frame, width=84):
    """Resize frame to given width, preserving aspect ratio (height auto-scaled)"""
    h, w = frame.shape[:2]
    new_w = width
    new_h = int(h * (new_w / w))
    resized = cv2.resize(frame, (new_w, new_h))
    return resized


def ocr_distance(frame):
    """Extract distance from frame via OCR. Return float distance in meters."""
    h, w = frame.shape[:2]

    # ADJUST THESE CROP COORDINATES TO YOUR SCREEN
    x1, x2 = 1800, 2200
    y1, y2 = 20, 60

    # Fix bounds
    x1 = max(0, min(x1, w))
    x2 = max(0, min(x2, w))
    y1 = max(0, min(y1, h))
    y2 = max(0, min(y2, h))

    if x2 <= x1 or y2 <= y1:
        print("OCR: Invalid crop region")
        return 0.0

    roi = frame[y1:y2, x1:x2]

    # Only show if valid
    if roi.size > 0:
        cv2.imshow("OCR Region", roi)
        cv2.waitKey(1)

    # TODO: Add OCR here later
    return 0.0

stream the screen though adb

In [4]:
import subprocess
import time
import cv2

# Start scrcpy in background
scrcpy_proc = subprocess.Popen([
    'scrcpy', f'--v4l2-sink={VIDEO_DEVICE}',
    '--no-window', '--max-size', '720', '--stay-awake'
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

time.sleep(2)

cap = cv2.VideoCapture(VIDEO_DEVICE)
if not cap.isOpened():
    raise RuntimeError(f"Cannot open {VIDEO_DEVICE}")

test: ocr distance from screen using `easyocr`

In [5]:
from PIL import Image
import easyocr
reader = easyocr.Reader(['en'], gpu=True)

def ocr_distance(frame):
    """Extract distance from frame via OCR. Return float distance in meters."""
    h, w = frame.shape[:2]

    # ADJUST THESE CROP COORDINATES TO YOUR SCREEN
    x1, y1 = 240, 24
    x2, y2 = x1 + 70, y1 + 19

    # Fix bounds
    x1 = max(0, min(x1, w))
    x2 = max(0, min(x2, w))
    y1 = max(0, min(y1, h))
    y2 = max(0, min(y2, h))

    if x2 <= x1 or y2 <= y1:
        return 0

    roi = frame[y1:y2, x1:x2]
    if roi.size == 0:
        return 0

    # Preprocessing for better OCR:
    # Preprocessing for better OCR:
    # Preprocessing for better OCR:
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

    # Display (for debugging in notebook)
    display(Image.fromarray(thresh, 'L'))

    # Run EasyOCR on preprocessed image
    try:
        result = reader.readtext(thresh, detail=0, allowlist='0123456789m')
        if result:
            text = result[0].strip().lower().replace('m', '')
            if text.isdigit():
                print("OCR Distance: ", text)
                return int(text)
    except Exception as e:
        pass
    return 0


while True:
    clear_output()

    # # ret, frame = cap.read()
    frame = get_frame(cap)
    # rframe = resize_img(frame)
    # Image.fromarray(frame)
    # Image.fromarray(rframe)
    ocr_distance(frame)
    # len(frame)
    time.sleep(50 / 1000)


KeyboardInterrupt: 

test: ocr using PaddleOCR

test: ocr using template

In [8]:
# load 0-9 character templates, for ocr'ing the distance from screen

import os


def load_chr_templates():
    templates = {}
    for i in range(10):
        path = f"assets/chr_{i}.png"
        if os.path.exists(path):
            img = cv2.imread(path, cv2.IMREAD_UNCHANGED)  # loads BGRA
            templates[str(i)] = img
    return templates


chr_templates = load_chr_templates()

In [81]:
def ocr_distance(frame):
    h, w = frame.shape[:2]
    x1, y1 = 240, 24
    x2, y2 = x1 + 70, y1 + 19

    x1 = max(0, min(x1, w))
    x2 = max(0, min(x2, w))
    y1 = max(0, min(y1, h))
    y2 = max(0, min(y2, h))

    if x2 <= x1 or y2 <= y1:
        return 0

    roi = frame[y1:y2, x1:x2]
    if roi.size == 0:
        return 0

    # Convert full ROI to grayscale (once)
    display(Image.fromarray(roi))

    text = ""
    pos = 0
    while pos < roi.shape[1]:
        best_char = None
        best_response = -1
        for char, tmpl in chr_templates.items():
            if tmpl is None or tmpl.shape[2] != 4:
                continue
            h_t, w_t = tmpl.shape[:2]
            if pos + w_t > roi.shape[1] or h_t != roi.shape[0]:
                continue

            # Extract patch from ROI (no modification)
            patch = roi[:, pos:pos + w_t]
            if patch.shape != (h_t, w_t):
                continue

            # Extract template grayscale and alpha
            tmpl_bgr = tmpl[:, :, :3]
            tmpl_alpha = tmpl[:, :, 3].astype(np.float32) / 255.0  # [0,1]

            tmpl_gray = cv2.cvtColor(tmpl_bgr, cv2.COLOR_BGR2GRAY).astype(np.float32)

            # Normalize both to [0,1]
            patch_norm = patch.astype(np.float32) / 255.0
            tmpl_norm = tmpl_gray / 255.0

            # Compute masked SSD: only where alpha > 0
            diff = (patch_norm - tmpl_norm) * tmpl_alpha
            ssd = np.sum(diff * diff)
            response = -ssd  # higher = better

            if response > best_response and ssd < 0.1:  # threshold tuned
                best_response = response
                best_char = char

        if best_char:
            text += best_char
            pos += chr_templates[best_char].shape[1]
        else:
            pos += 1  # move 1px if no match

    print('ocr:', text)
    if text.isdigit():
        return int(text)
    return 0


frame = get_frame(cap)
ocr_distance(frame)

ocr: 


0

run the model

In [306]:
current_action = None
current_hold_proc = None

timings = []
step = 0

try:
    while True:
        start_iter = time.perf_counter()

        # Get frame
        ret, frame = cap.read()
        if not ret:
            continue

        # Run model
        new_action = get_action_from_model(frame)

        # Execute action if changed
        if new_action != current_action:
            if current_hold_proc is not None:
                current_hold_proc.terminate()
                current_hold_proc = None

            if new_action == "accel":
                # 1ms tap
                subprocess.run([
                    'adb', 'shell', 'input', 'swipe',
                    str(ACCEL_POS[0]), str(ACCEL_POS[1]),
                    str(ACCEL_POS[2]), str(ACCEL_POS[3]),
                    str(TAP_DURATION_MS)
                ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                # Long hold
                current_hold_proc = subprocess.Popen([
                    'adb', 'shell', 'input', 'swipe',
                    str(ACCEL_POS[0]), str(ACCEL_POS[1]),
                    str(ACCEL_POS[2]), str(ACCEL_POS[3]),
                    str(HOLD_DURATION_MS)
                ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

            elif new_action == "brake":
                # 1ms tap
                subprocess.run([
                    'adb', 'shell', 'input', 'swipe',
                    str(BRAKE_POS[0]), str(BRAKE_POS[1]),
                    str(BRAKE_POS[2]), str(BRAKE_POS[3]),
                    str(TAP_DURATION_MS)
                ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                # Long hold
                current_hold_proc = subprocess.Popen([
                    'adb', 'shell', 'input', 'swipe',
                    str(BRAKE_POS[0]), str(BRAKE_POS[1]),
                    str(BRAKE_POS[2]), str(BRAKE_POS[3]),
                    str(HOLD_DURATION_MS)
                ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

            current_action = new_action

        # Timing
        iter_time = time.perf_counter() - start_iter
        timings.append(iter_time)
        if len(timings) > 100:
            timings = timings[-100:]
        avg_time = sum(timings) / len(timings)
        fps = 1.0 / avg_time if avg_time > 0 else 0

        print(f"took: {iter_time * 1000:.1f} ms | avg took: {avg_time * 1000:.1f} ms | fps: {fps:.1f}")

        step += 1

except KeyboardInterrupt:
    if current_hold_proc is not None:
        current_hold_proc.terminate()
    cap.release()

took: 143.8 ms | avg took: 143.8 ms | fps: 7.0
took: 127.7 ms | avg took: 135.7 ms | fps: 7.4
took: 21.3 ms | avg took: 97.6 ms | fps: 10.2
took: 22.8 ms | avg took: 78.9 ms | fps: 12.7
took: 60.8 ms | avg took: 75.3 ms | fps: 13.3
took: 52.9 ms | avg took: 71.6 ms | fps: 14.0
took: 177.2 ms | avg took: 86.6 ms | fps: 11.5
took: 20.4 ms | avg took: 78.4 ms | fps: 12.8
took: 128.6 ms | avg took: 83.9 ms | fps: 11.9
took: 20.5 ms | avg took: 77.6 ms | fps: 12.9
took: 144.6 ms | avg took: 83.7 ms | fps: 11.9
took: 20.6 ms | avg took: 78.4 ms | fps: 12.7
took: 20.5 ms | avg took: 74.0 ms | fps: 13.5
took: 118.9 ms | avg took: 77.2 ms | fps: 13.0
took: 20.4 ms | avg took: 73.4 ms | fps: 13.6
took: 20.4 ms | avg took: 70.1 ms | fps: 14.3
took: 114.8 ms | avg took: 72.7 ms | fps: 13.8
took: 20.4 ms | avg took: 69.8 ms | fps: 14.3
took: 119.7 ms | avg took: 72.4 ms | fps: 13.8
took: 20.6 ms | avg took: 69.8 ms | fps: 14.3
took: 114.7 ms | avg took: 72.0 ms | fps: 13.9
took: 125.4 ms | avg took

---
# Making the model
---

In [301]:
import torch
import torch.nn as nn


# === LOAD YOUR MODEL HERE ===
class HillClimbRacerV1(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32 * 19 * 7, 128),  # input size matches 34x83 → (34-5)//2+1=15 → wait, recalc properly
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        return self.net(x)


# Initialize model
device = torch.device('cuda')
model = HillClimbRacerV1().to(device)
model.eval()


# If you have a saved model, load it:
# model.load_state_dict(torch.load('your_model.pth', map_location=device))

def preprocess_frame(frame):
    """Convert raw frame to model input tensor (1, 1, H, W)"""
    # Example: crop, resize, grayscale, normalize
    # ADJUST CROP/RESIZE TO YOUR GAME UI
    cropped = frame[100:900, 200:2200]  # adjust based on your screen
    resized = cv2.resize(cropped, (84, 84))
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    tensor = torch.from_numpy(gray).float() / 255.0
    tensor = tensor.unsqueeze(0).unsqueeze(0)  # (1, 1, 84, 84)
    return tensor.to(device)


def get_action_from_model(frame):
    tensor = preprocess_frame(frame)
    with torch.no_grad():
        logits = model(tensor)
        pred = torch.argmax(logits, dim=1).item()
    actions = ["accel", "brake", "none"]
    return actions[pred]